In [1]:
# Dependencies 
from docplex.mp.model import Model 
import numpy as np
import pandas as pd

### Exercise 10.4

In [25]:
# Create model 
model = Model()

# Create decision vars 
parts, years = ['a', 'b', 'c', 'd', 'e'], [0, 1, 2, 3]

x = {'a': model.integer_var_dict(years, name = 'x_a', lb = 0), 
     'b': model.integer_var_dict(years, name = 'x_b', lb = 0), 
     'c': model.integer_var_dict(years, name = 'x_c', lb = 0), 
     'd': model.integer_var_dict(years, name = 'x_d', lb = 0), 
     'e': model.integer_var_dict(years, name = 'x_e', lb = 0)}

y = {'a': model.integer_var_dict(years, name = 'y_a', lb = 0), 
     'b': model.integer_var_dict(years, name = 'y_b', lb = 0), 
     'c': model.integer_var_dict(years, name = 'y_c', lb = 0), 
     'd': model.integer_var_dict(years, name = 'y_d', lb = 0), 
     'e': model.integer_var_dict(years, name = 'y_e', lb = 0)}

# Demands 
demands = {'a': dict(zip(years, [0, 5, 2, 3])), 
           'b': dict(zip(years, [0, 4, 1, 4])), 
           'c': dict(zip(years, [0, 4, 1, 3])), 
           'd': dict(zip(years, [0, 2, 7, 0])), 
           'e': dict(zip(years, [0, 1, 0, 2]))}

# Costs
costs = {'a': dict(zip(years, [0, 0.5, 0.6, 0.7])), 
         'b': dict(zip(years, [0, 2.0, 2.2, 2.5])), 
         'c': dict(zip(years, [0, 5.0, 5.5, 6.0])), 
         'd': dict(zip(years, [0, 1.0, 1.1, 1.3])), 
         'e': dict(zip(years, [0, 7.8, 7.5, 7.0]))}


# Demand constraints
demand_cts = []
for year in [1,2,3]:
    for part in parts[:-1]:
        new_const = x[part][year] + x['e'][year] + y[part][year - 1]  >= demands[part][year] + demands['e'][year]
        demand_cts.append(new_const)
    
    
# Balance constraints 
balance_cts = []
for year in [1,2]: 
    for part in parts[:-1]:
        new_const = x[part][year] + x['e'][year] + y[part][year - 1] - (demands[part][year] + demands['e'][year]) == y[part][year]
        balance_cts.append(new_const)
    
# Add constraints to model
model.add_constraints(demand_cts)
model.add_constraints(balance_cts)
    
# Add constraints for modeling purposes
model.add_constraints([x[part][0] == 0 for part in parts])   
model.add_constraints([y[part][0] == 0 for part in parts])  

# Objective 
obj = np.sum([costs[part][year] * x[part][year] for year in years[1:] for part in parts])
model.minimize(obj)

model.solve()
model.print_solution()

objective: 87.000
  x_a_1=2
  x_b_1=1
  x_d_1=1
  x_e_1=5
  x_e_2=4
  x_e_3=2
  y_a_1=1
  y_a_2=3
  y_b_1=1
  y_b_2=4
  y_c_2=3
  y_d_1=3


### Knapsack Problem

In [5]:
def knapsack(capacity: int, weights: list, profits: list, n_items: int): 
    
    # Check base condition where we can't do anything
    if (n_items == 0) or (capacity == 0):
        return 0
    
    if weights[n_items - 1] <= capacity: 
        option_1 = profits[n_items - 1] + knapsack(capacity - weights[n_items - 1], weights, profits, n_items - 1)
        option_2 = knapsack(capacity, weights, profits, n_items - 1)
        return max(option_1, option_2)
    
    else: 
        return knapsack(capacity, weights, profits, n_items - 1)

In [6]:
# Run 
profits = [20, 18, 10, 8, 4, 1]
weights = [5, 7, 4, 3, 2, 1]
capacity = 11
n_items = len(profits)

print(f'Maximum profit at {knapsack(11, [5, 7, 4, 3, 2, 1], [20, 18, 10, 8, 4, 1], 6)}')

Maximum profit at 34


We see this comes from the x1, x3, x5 = 1 and x2, x4, x6 = 0.